In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

In [4]:
train= pd.read_csv('C:/datasets/train_sales.csv')
test = pd.read_csv('C:/datasets/test_sales.csv')

In [5]:
train_ds = train.groupby(['date_block_num', 'location','shop_type','shop_id','item_category_id','item_category', 'subcat','item_id',]).agg({'item_cnt_day':'sum','item_price':'mean'}).reset_index()
train_ds.columns = ['date_block_num', 'location','shop_type','shop_id','item_category_id','item_category','subcat','item_id','item_cnt_monthly','mean_item_price']

In [6]:
def label_encode_data(data):
    
    #shop
    data["loc_cd"] = LabelEncoder().fit_transform(data["location"])
    data["shop_type_cd"] = LabelEncoder().fit_transform(data["shop_type"])
    data.drop(["location"], axis=1, inplace=True)
    data.drop(["shop_type"], axis=1, inplace=True)

    #item
    data["item_cat_cd"] = LabelEncoder().fit_transform(data["item_category"])
    data["subcat_cd"] = LabelEncoder().fit_transform(data["subcat"])
    data.drop(["item_category"], axis=1, inplace=True)
    data.drop(["subcat"], axis=1, inplace=True)
    
    return data

def one_hot_encode_data(data):
    
    data = pd.get_dummies(data, columns = ['location', 'shop_type','item_category','subcat'])
    
    return data

def mean_encode_data(data):
    Mean_encoded_loc = data.groupby(['location'])['item_cnt_monthly'].mean().to_dict()
    data['loc_cd'] =  data['location'].map(Mean_encoded_loc)
    Mean_encoded_shop = data.groupby(['shop_type'])['item_cnt_monthly'].mean().to_dict()
    data['shop_type_cd'] =  data['shop_type'].map(Mean_encoded_shop)
    data.drop(["location"], axis=1, inplace=True)
    data.drop(["shop_type"], axis=1, inplace=True)

    #item
    Mean_encoded_cat= data.groupby(['item_category'])['item_cnt_monthly'].mean().to_dict()
    data['item_cat_cd'] =  data['item_category'].map(Mean_encoded_cat)
    Mean_encoded_subcat = data.groupby(['subcat'])['item_cnt_monthly'].mean().to_dict()
    data['subcat_cd'] =  data['subcat'].map(Mean_encoded_subcat)
    data.drop(["item_category"], axis=1, inplace=True)
    data.drop(["subcat"], axis=1, inplace=True)
    
    return data

def novelty_feature(data):
    
    #novelty
    min_values = data.groupby("item_id")["date_block_num"].min().reset_index()
    min_values.columns = ['item_id','first_sales_date_block']
    data = pd.merge(data, min_values, on='item_id', how = 'left')
    
    return data

def lag_features(df, lags, col_list):
    
    for col_name in col_list:
        tmp = df[["date_block_num", "shop_id", "item_id", col_name]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = [
                "date_block_num",
                "shop_id",
                "item_id",
                col_name + "_lag_" + str(i),
            ]
            shifted["date_block_num"] += i
            df = pd.merge(
                df, shifted, on=["date_block_num", "shop_id", "item_id"], how="left"
            )
    return df

def last_halfyear_feathure(train_ds):
#last 6 month average of sales
    train_ds["last_6month_cnt"] = train_ds[["item_cnt_monthly_lag_1", "item_cnt_monthly_lag_2", "item_cnt_monthly_lag_3", "item_cnt_monthly_lag_4", "item_cnt_monthly_lag_5","item_cnt_monthly_lag_6"]].mean(skipna=True, axis=1)
    return train_ds

In [7]:
def primary_data_for_modeling(data, enc):
    
    train_ds = novelty_feature(data)
    if enc=='OHE':
        train_ds = one_hot_encode_data(train_ds)
    if enc=='mean':
        train_ds = mean_encode_data(train_ds)
    if enc=='label':
        train_ds = label_encode_data(train_ds)
    train_ds = lag_features(train_ds, [1, 2, 3, 4, 5, 6, 12], ["item_cnt_monthly"])
    train_ds = last_halfyear_feathure(train_ds)
    train_ds.fillna(0, inplace=True)
    
    return train_ds

Here i changed cross-validation function for it to fit as a parameter of cross-validation of grid search.

The whole structure of it is the same except the output is yield indices.

In [30]:
def cross_validation_for_grid(data, month):
    
    if month>=data.date_block_num.max():
        print("Cannot be splited")
        
    elif month<=data.date_block_num.min():
        print("Cannot be splited")
    else:
        while True:
            
            ##feature gen.
            data_1 = data[:(data[data.date_block_num==(month+1)][-1:].index[0]+1)]
            #data_1 = primary_data_for_modeling(data_1, enc)
            
            train_ds = np.array(data_1[:(data_1[data_1.date_block_num==month][-1:].index[0]+1)].index)
            if (month+1)<=data_1.date_block_num.max():
                test_ds = np.array(data_1[(data_1[data_1.date_block_num==month][-1:].index[0]+1):(data_1[data_1.date_block_num==(month+1)][-1:].index[0]+1)].index)
            else:
                test_ds = np.array(data_1[(data_1[data_1.date_block_num==month][-1:].index[0]+1):].index)
            
            yield train_ds, test_ds

            month+=1
            
            if month>=data.date_block_num.max():
                break

Next there is a function that do the hyperparameters searching since in the task it is said to experiment with models(?).

In [31]:
def grid_search_func(train_ds, model, param_space, enc):
    
            from sklearn.model_selection import GridSearchCV

            tscv = cross_validation_for_grid(train_ds, 28)
            #train_ds = primary_data_for_modeling(train_ds, enc)
            
            best_params_models = {}
            
            grid_search = GridSearchCV(model, param_space, cv=cross_validation_for_grid(train_ds, 28), return_train_score=True)
            grid_search.fit(train_ds.drop('item_cnt_monthly', axis=1), train_ds['item_cnt_monthly'])
                
            print(model)
            print("Best params: ", grid_search.best_params_)
            print("Best score: ", grid_search.best_score_)

In [33]:
train_ds_pred = primary_data_for_modeling(train_ds, 'label')

In [39]:
train_ds_pred = train_ds[train_ds.date_block_num>11]

In [40]:
train_ds_pred = primary_data_for_modeling(train_ds_pred, 'label')

In [41]:
train_ds_pred

,date_block_num,shop_id,item_category_id,item_id,item_cnt_monthly,mean_item_price,first_sales_date_block,loc_cd,shop_type_cd,item_cat_cd,subcat_cd,item_cnt_monthly_lag_1,item_cnt_monthly_lag_2,item_cnt_monthly_lag_3,item_cnt_monthly_lag_4,item_cnt_monthly_lag_5,item_cnt_monthly_lag_6,item_cnt_monthly_lag_12,last_6month_cnt
0,12,2,2,5572,2.0,1589.85,12,0,5,0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,12,2,2,5608,3.0,799.00,12,0,5,0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,12,2,2,5613,2.0,6089.85,12,0,5,0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,12,2,2,5622,4.0,549.00,12,0,5,0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,12,2,2,5630,1.0,1798.00,12,0,5,0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920867,33,59,75,5383,1.0,4390.00,32,30,5,14,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
920868,33,59,79,17717,13.0,802.55,12,30,5,15,53,12.0,16.0,24.0,10.0,13.0,19.0,21.0,15.666667
920869,33,59,83,22087,6.0,119.00,12,30,5,18,63,3.0,2.0,5.0,1.0,2.0,2.0,6.0,2.500000
920870,33,59,83,22088,2.0,119.00,12,30,5,18,63,1.0,7.0,7.0,4.0,3.0,3.0,7.0,4.166667


**Decision Tree:**

In [42]:
param_DT = {'max_depth': [3, 5, 6, 8],}
grid_search_func(train_ds_pred, DecisionTreeRegressor(), param_DT, 'label')

DecisionTreeRegressor()
Best params:  {'max_depth': 3}
Best score:  0.5041126418759856


**Random Forest:**

In [ ]:
param_RF = {'max_depth': [3, 5, 6, 8],
           #'n_estimators': [100, 150, 200],
           }
grid_search_func(train_ds_pred, RandomForestRegressor(), param_RF, 'label')

**SVG:**

In [ ]:
param_SVG = {'C': [0.1, 1, 10],
             'kernel': ['rbf', 'linear'],}
grid_search_func(train_ds_pred, SVR(), param_SVG, 'label')

**XGBoost:**

In [ ]:
param_XGB = {'max_depth': [3, 5, 6, 8],
             'n_estimators': [100, 150, 200],}
grid_search_func(train_ds_pred, xgb.XGBRegressor(), param_XGB, 'label')